# Mangrove Loss: 1988 - 2024
The code in this file creates an HTML file that visualizes the loss in mangrove forest coverage from 1988 to 2024 on an interactive Earth Engine map. It adds individual layers for each year of the mangrove extent, allowing the user to view and compare areas of mangrove loss. 

In [ ]:
# --- Run underlying python scripts
import GEN01_GEE_Authenticate
import GEN02_AOI as  aoi_def
import GEN03_helper_functions as HF
import GEN04_mangrove_layers as ML 
import GEN05_HTML_layout as HTML 
import GEN06_shapefiles as shp


In [4]:
import ee, os


# SERVICE_ACCOUNT = os.environ.get("GEE_SERVICE_ACCOUNT")
# KEY_FILE = "key.json"   # GitHub Actions writes the secret here

# credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
# ee.Initialize(credentials)


ee.Authenticate()
ee.Initialize(project="gee-mekong-map")

In [5]:
# ------------- Packages & GEE ------------
import ee                                       # Google Earth Engine API for Python
import geemap.foliumap as geemap_folium         # Geemap module integrating Folium maps
import folium                                   # Interactive mapping library based on Leaflet.js
import geemap                                   # Toolkit for working with Google Earth Engine in Python
from geemap import cartoee                      # Module for exporting and visualizing Earth Engine maps using Matplotlib
import ipyleaflet                               # Interactive maps in Jupyter using Leaflet
import ipywidgets as widgets                    # Interactive UI controls for Jupyter notebooks
from branca.element import Element              # Low-level HTML/JS elements for Folium/Branca maps
import geopandas as gpd                         # Spatial data handling (GeoDataFrames)
import glob                                     # File pattern matching (e.g., list all .tif files in a folder)
import json                                     # JSON encoding and decoding (read/write .json files)


In [ ]:
# Mangrove loss = 2010 mangroves not present in 2015
# ------------- Calculate mangrove loss per time interval -------------
loss_1988_1992 = ML.mangrove_1988.And(ML.mangrove_1992.Not())
loss_1992_1997 = ML.mangrove_1992.And(ML.mangrove_1997.Not())
loss_1997_2001 = ML.mangrove_1997.And(ML.mangrove_2001.Not())
loss_2001_2005 = ML.mangrove_2001.And(ML.mangrove_2005.Not())
loss_2005_2010 = ML.mangrove_2005.And(ML.mangrove_2010.Not())
loss_2010_2015 = ML.mangrove_2010.And(ML.mangrove_2015.Not())
loss_2015_2020 = ML.mangrove_2015.And(ML.mangrove_2020.Not())
loss_2020_2024 = ML.mangrove_2020.And(ML.mangrove_2024.Not())

geemap_folium.ee_initialize = lambda *args, **kwargs: None


Map = geemap_folium.Map(center=[9.2, 105.75], zoom=11, initialize=False)


Map.add_basemap('SATELLITE')



In [7]:
# ------------- Add loss layers with palette and generate tile URLs -------------
loss_layers = {
    "Mangrove loss (1988-1992)": (loss_1988_1992, ML.color_1988_1992_loss),
    "Mangrove loss (1992-1997)": (loss_1992_1997, ML.color_1992_1997_loss),
    "Mangrove loss (1997-2001)": (loss_1997_2001, ML.color_1997_2001_loss),
    "Mangrove loss (2001-2005)": (loss_2001_2005, ML.color_2001_2005_loss),
    "Mangrove loss (2005-2010)": (loss_2005_2010, ML.color_2005_2010_loss),
    "Mangrove loss (2010-2015)": (loss_2010_2015, ML.color_2010_2015_loss),
    "Mangrove loss (2015-2020)": (loss_2015_2020, ML.color_2015_2020_loss),
    "Mangrove loss (2020-2024)": (loss_2020_2024, ML.color_2020_2024_loss),
}

# -------------- Add loss maps -----------------
Map.addLayer(loss_1988_1992.updateMask(loss_1988_1992), {"palette": [ML.color_1988_1992_loss]}, "Mangrove loss (1988-1992)")
Map.addLayer(loss_1992_1997.updateMask(loss_1992_1997), {"palette": [ML.color_1992_1997_loss]}, "Mangrove loss (1992-1997)")
Map.addLayer(loss_1997_2001.updateMask(loss_1997_2001), {"palette": [ML.color_1997_2001_loss]}, "Mangrove loss (1997-2001)")
Map.addLayer(loss_2001_2005.updateMask(loss_2001_2005), {"palette": [ML.color_2001_2005_loss]}, "Mangrove loss (2001-2005)")
Map.addLayer(loss_2005_2010.updateMask(loss_2005_2010), {"palette": [ML.color_2005_2010_loss]}, "Mangrove loss (2005-2010)")
Map.addLayer(loss_2010_2015.updateMask(loss_2010_2015), {"palette": [ML.color_2010_2015_loss]}, "Mangrove loss (2010-2015)")
Map.addLayer(loss_2015_2020.updateMask(loss_2015_2020), {"palette": [ML.color_2015_2020_loss]}, "Mangrove loss (2015-2020)")
Map.addLayer(loss_2020_2024.updateMask(loss_2020_2024), {"palette": [ML.color_2020_2024_loss]}, "Mangrove loss (2020-2024)")


In [8]:
# for name, (layer, color) in loss_layers.items():
#     vis = {"palette": [color]}
#     Map.addLayer(layer.updateMask(layer), vis, name)
#     tile_urls[name] = generate_tile_url(layer, vis)

# # Save tile URLs to JSON
# with open("tile_urls.json", "w") as f:
#     json.dump(tile_urls, f, indent=2)
  


Add legend, scalebar, north arrow, Living Lab logo

In [9]:
# ------- Add legend      --------
legend_title = "Mangrove Loss Map (1988-2024)"
HTML.add_folium_legend(Map, legend_title, HTML.legend_dict_mangrove_LOSS, style=HTML.style)

# ------- Add scalebar     -------
Map.add_child(HTML.ScaleBar(font_size="14px"))

# ------- Add north arrow --------
HTML.add_north_arrow(Map, position="bottomleft", arrow_size="35px", text_size="25px")

# ------- Add Living LAb logo -----
Map.get_root().html.add_child(folium.Element(HTML.logo_html))

Add shapefiles

In [10]:
# Add the commune boundaries to the map
folium.GeoJson(
    shp.geojson_commune,
    name="Commune boundaries",
    style_function=shp.commune_style,
    ).add_to(Map)


#  Add the sea dikes group to the map
shp.sea_dikes_group.add_to(Map)


# Add the breakwater group to the map
shp.breakwaters_group.add_to(Map)

# other human activities
shp.Extra_group.add_to(Map)
folium.LayerControl().add_to(Map)   

### Create the HTML file

In [ ]:
Map.to_html("Mangrove_LOSS_map.html")